<a href="https://colab.research.google.com/github/akshay4n/Quantitaive-Research/blob/main/equalWeight_S%26P500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 4.3 MB/s 


In [4]:
#import
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [5]:
# %cd /content
path = '/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/sp_500_stocks.csv'

In [6]:
stocks = pd.read_csv(path)

In [7]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [8]:
#Acquire API Token
# !cp /content/drive/MyDrive/QuantitativeResearch/EqualWeight_S&P500/secrets_imp.py /content.

import sys
import os

py_file_location = "/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/secrets_imp.py"
sys.path.append(os.path.abspath(py_file_location))

%cd /content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500
!pwd
!ls

/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500
/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500
__pycache__  secrets_imp.py  sp_500_stocks.csv


In [9]:
# import secrets_imp
from secrets_imp import IEX_CLOUD_API_TOKEN

In [10]:
#info reqd - Market capitalization of each stock, Price of each stock
symbol ='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 67921338, 'calculationPrice': 'close', 'change': -0.4, 'changePercent': -0.00235, 'close': 0, 'closeSource': 'fifaclio', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 180.18, 'iexCloseTime': 1716420928445, 'iexLastUpdated': 1695911649670, 'iexMarketPercent': 0.002083936451304491, 'iexOpen': 179.57, 'iexOpenTime': 1736747004596, 'iexRealtimePrice': 175.4, 'iexRealtimeSize': 51, 'iexVolume': 2263, 'lastTradeTime': 1698308582615, 'latestPrice': 177.08, 'latestSource': 'Close', 'latestTime': 'August 18, 2022', 'latestUpdate': 1694251848853, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2818575280187, 'oddLotDelayedPrice': None, 'oddLotDel

In [11]:
price = data['latestPrice']
# print(price)
market_cap = data['marketCap']
# print(market_cap)

In [12]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_df = pd.DataFrame(columns=my_columns)
# final_df

final_df.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
         index = my_columns
        ),
        ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,177.08,2818575280187,N/A


In [13]:
final_df = pd.DataFrame(columns=my_columns)
i=0
for stock in stocks['Ticker'][:135]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    # i+=1
    # print(i,"\n")
    data = requests.get(api_url).json()
    final_df = final_df.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = my_columns
        ),
        ignore_index=True
    )

In [14]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,144.46,43210727554,N/A
1,AAL,15.16,10066920109,N/A
2,AAP,213.58,12893131862,N/A
3,AAPL,180.92,2858634511204,N/A
4,ABBV,146.80,259067734454,N/A
...,...,...,...,...
130,DG,270.43,58863618477,N/A
131,DGX,137.01,16092272597,N/A
132,DHI,80.27,27993488149,N/A
133,DHR,297.96,223906838900,N/A


In [15]:
#using Batch API Calls to improve performance
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [16]:
symbol_groups = list(chunks(stocks['Ticker'][1:135],100))
# symbol_groups = symbol_groups+(list(chunks(stocks['Ticker'][137:],100)))
print(symbol_groups)

[1        AAL
2        AAP
3       AAPL
4       ABBV
5        ABC
       ...  
96        CL
97       CLX
98       CMA
99     CMCSA
100      CME
Name: Ticker, Length: 100, dtype: object, 101     CMG
102     CMI
103     CMS
104     CNC
105     CNP
106     COF
107     COG
108     COO
109     COP
110    COST
111    COTY
112     CPB
113    CPRT
114     CRM
115    CSCO
116     CSX
117    CTAS
118     CTL
119    CTSH
120    CTVA
121    CTXS
122     CVS
123     CVX
124     CXO
125       D
126     DAL
127      DD
128      DE
129     DFS
130      DG
131     DGX
132     DHI
133     DHR
134     DIS
Name: Ticker, dtype: object]


In [17]:
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
        # api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
        # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_strings}&types=quote/&token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
            )
            ,ignore_index = True
        )

In [18]:
#Calculate number of shares to Buy
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter a valid value')

Enter the value of your portfolio: 10000000
10000000.0


In [20]:
position_size = val/len(final_df.index)

for i in range(0,len(final_df.index)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i,'Stock Price'])

final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAL,15.37,9995699607,4855
1,AAP,208.36,13082694753,358
2,AAPL,178.42,2839527081756,418
3,ABBV,142.83,255021481487,522
4,ABC,150.03,31784927784,497
...,...,...,...,...
129,DG,267.64,61319028108,278
130,DGX,137.52,16408976085,542
131,DHI,79.58,27579691450,937
132,DHR,307.78,214081394996,242


In [25]:
#write output to file
writer = pd.ExcelWriter('recommendedTrades.xlsx',engine='xlsxwriter')
final_df.to_excel(writer,'Recommended Trades',index=False)
writer.save()